# 골드 티어 달성을 위한  League of Legend API 분석

#### 1. Riot Api 획득 및 적용 
+ https://developer.riotgames.com/ 에서 api key 획득 가능 

In [16]:
import requests
import json
import pandas as pd
import numpy as np
from time import sleep
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

### 1. 닉네임 검색을 통한 정보 검색

#### https://developer.riotgames.com/app/517336/apis 참고

In [3]:
apiDefault = {
    'region': 'RGAPI-44052313-9ba2-4401-8e9b-e46f82ca3430',  # 한국서버
    'key': 'RGAPI-57b165cc-ecd7-4302-a1bd-4c3cd1eb801f',  # api key
    'summonerName': '뱀꼬치',  # 인게임 닉네임
}

# api 호출시 사용되는 url
url = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/%EB%B1%80%EA%BC%AC%EC%B9%98?api_key=" + apiDefault['key']

# get method 를 통해 api 호출
r = requests.get(url)

r.json()

{'id': 'zCg-Fo_0oVscbgtb1g-B_A5e6M-bPgl286uskb12th5TwQ',
 'accountId': 'uKu9_3GADEi9gSVwUPYi4aA5bZRhMSMAgTRTSsLx5oYt',
 'puuid': 'VsaPcKfnhHhusZcXkYwSsNahibTH5HLnMJsnB1voKG0tH5OC03i_j2cMsaBj3d3ha5HAAxtALQ7yPw',
 'name': '뱀꼬치',
 'profileIconId': 4573,
 'revisionDate': 1626709186521,
 'summonerLevel': 374}

### 2. 현재 계정 리그 정보 호출

In [4]:
url = 'https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/zCg-Fo_0oVscbgtb1g-B_A5e6M-bPgl286uskb12th5TwQ?api_key=' + apiDefault['key']
r = requests.get(url)

# 솔로랭크 정보만 추출하여 Dict형으로 저장 
info_data = r.json()[1]

# # DataFrame으로 변환
info_data = pd.Series(info_data, index = info_data.keys())
info_data = pd.DataFrame(info_data).T

# # 필요없는 행 삭제
info_data.drop(['leagueId', 'summonerId', 'veteran', 'inactive', 'freshBlood', 'hotStreak'], axis = 1, inplace = True)

# # 승률 컬럼 추가 
info_data['win_rate(%)'] = (info_data['wins'] / (info_data['wins'] + info_data['losses'])) *100

info_data

queueType    tier rank summonerName leaguePoints wins losses  \
0  RANKED_SOLO_5x5  SILVER  III          뱀꼬치           16  343    361   

  win_rate(%)  
0     48.7216

In [5]:
# 승, 패 int로 수정 
info_data['wins'] = int(info_data['wins']) 
info_data['wins']
info_data['losses'] = int(info_data['losses'])
info_data['losses']

0    361
Name: losses, dtype: int64

In [6]:
# info_data로 확인한 결과 13시즌 총 진행 판수 704판 진행 확인 game_data로 gameId 및 기타 내용 저장
url = ['https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/uKu9_3GADEi9gSVwUPYi4aA5bZRhMSMAgTRTSsLx5oYt?queue=420&season=13&api_key=' + apiDefault['key'],
       'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/uKu9_3GADEi9gSVwUPYi4aA5bZRhMSMAgTRTSsLx5oYt?queue=420&season=13&beginIndex=101&api_key=' + apiDefault['key'], 
       'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/uKu9_3GADEi9gSVwUPYi4aA5bZRhMSMAgTRTSsLx5oYt?queue=420&season=13&beginIndex=201&api_key=' + apiDefault['key'], 
       'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/uKu9_3GADEi9gSVwUPYi4aA5bZRhMSMAgTRTSsLx5oYt?queue=420&season=13&beginIndex=301&api_key=' + apiDefault['key'], 
       'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/uKu9_3GADEi9gSVwUPYi4aA5bZRhMSMAgTRTSsLx5oYt?queue=420&season=13&beginIndex=401&api_key=' + apiDefault['key'], 
       'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/uKu9_3GADEi9gSVwUPYi4aA5bZRhMSMAgTRTSsLx5oYt?queue=420&season=13&beginIndex=501&api_key=' + apiDefault['key'], 
       'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/uKu9_3GADEi9gSVwUPYi4aA5bZRhMSMAgTRTSsLx5oYt?queue=420&season=13&beginIndex=601&api_key=' + apiDefault['key']] 

url_list = []
for i in range(len(url)):
    r = requests.get(url[i])
    url_list.append(r.json())

In [8]:
pre = []
for i in range(len(url_list)):
    pre.append(url_list[i])

a = pre[0]['matches'] + pre[1]['matches'] + pre[2]['matches'] + pre[3]['matches'] + pre[4]['matches'] + pre[5]['matches'] +pre[6]['matches']
game_data = pd.DataFrame(a)

pd.options.display.max_rows = 700
game_data


platformId      gameId  champion  queue  season      timestamp  \
0           KR  5334404274        35    420      13  1626673994082   
1           KR  5334242624        56    420      13  1626663114292   
2           KR  5333610758       235    420      13  1626621083921   
3           KR  5333454235        81    420      13  1626618959831   
4           KR  5332976006       887    420      13  1626606146954   
5           KR  5332961992        82    420      13  1626604901684   
6           KR  5332521087       517    420      13  1626592689153   
7           KR  5331486384        35    420      13  1626543060469   
8           KR  5331413637       131    420      13  1626541188392   
9           KR  5331308088        78    420      13  1626538972771   
10          KR  5331370333       202    420      13  1626536792846   
11          KR  5331253212       145    420      13  1626534643704   
12          KR  5331116012        96    420      13  1626532492394   
13          KR  5330998829        18    420      13  1626530658031   
14          KR  5330658751        61    420      13  1626527784340   
15          KR  5325909418        19    420      13  1626344759137   
16          KR  5325869354        18    420      13  1626340217966   
17          KR  5325801969       412    420      13  1626336678664   
18          KR  5324063905       517    420      13  1626264211833   
19          KR  5323601571       136    420      13  1626246974544   
20          KR  5323443080       163    420      13  1626238676903   
21          KR  5321690587       163    420      13  1626169479609   
22          KR  5319918929        78    420      13  1626086895473   
23          KR  5318870398        18    420      13  1626021921870   
24          KR  5318707028        35    420      13  1626019448579   
25          KR  5318761641        67    420      13  1626017519027   
26          KR  5318645133        51    420      13  1626014931124   
27          KR  5318185394        56    420      13  1626004913511   
28          KR  5316167041        35    420      13  1625932768917   
29          KR  5315833362        56    420      13  1625924290233   
30          KR  5315742240        78    420      13  1625921664506   
31          KR  5312656919       145    420      13  1625823935749   
32          KR  5312603372        78    420      13  1625822304008   
33          KR  5311642689        78    420      13  1625758019038   
34          KR  5310390107        51    420      13  1625720723858   
35          KR  5309733206       136    420      13  1625678307902   
36          KR  5309720773        35    420      13  1625676344170   
37          KR  5309636131       517    420      13  1625674092703   
38          KR  5309358337        36    420      13  1625664717127   
39          KR  5308268781        78    420      13  1625632528706   
40          KR  5308264634        78    420      13  1625629392539   
41          KR  5306723375        56    420      13  1625556995181   
42          KR  5305064441        35    420      13  1625482126360   
43          KR  5304869075        35    420      13  1625480085572   
44          KR  5303441970       222    420      13  1625403010539   
45          KR  5303295587        81    420      13  1625401018737   
46          KR  5303227392       236    420      13  1625399141932   
47          KR  5303016963        67    420      13  1625393015378   
48          KR  5302972940       145    420      13  1625389115882   
49          KR  5302868167        78    420      13  1625387608424   
50          KR  5301494153       517    420      13  1625328037513   
51          KR  5301388471        78    420      13  1625324341270   
52          KR  5298736404        78    420      13  1625237144177   
53          KR  5296206585        56    420      13  1625139889763   
54          KR  5295608169        51    420      13  1625127770818   
55          KR  5295673821       234    420      13  1625126120620   
56          KR

In [28]:
# 최근 100게임 세부 데이터 불러오기
detail = []
detail2 = []
for i in tqdm(range(700)):
    url = F"https://kr.api.riotgames.com/lol/match/v4/matches/{game_data['gameId'][i]}?api_key={apiDefault['key']}"
    r = requests.get(url)
    b = r.json()
    for j in range(10):
        if b['participantIdentities'][j]['player']['summonerName'] == '뱀꼬치':
            detail.append(b['participants'][j])
        else:
            pass
    sleep(1)

100%|██████████| 700/700 [16:24<00:00,  1.41s/it]


In [29]:
a = detail

In [52]:
detail1 = []
for i in tqdm(range(700)):
    url = F"https://kr.api.riotgames.com/lol/match/v4/matches/{game_data['gameId'][i]}?api_key={apiDefault['key']}"
    req = requests.get(url)
    b = json.loads(req.text)
    if a[i]['stats']['win'] == True:
        if b['teams'][0]['win'] == 'Win':
            detail1.append(b['teams'][0])
        elif b['teams'][1]['win'] == 'Win':
            detail1.append(b['teams'][1])

    elif a[i]['stats']['win'] == False:
        if b['teams'][0]['win'] == 'Fail':
            detail1.append(b['teams'][0])
        elif b['teams'][1]['win'] == 'Fail':
            detail1.append(b['teams'][1])
    sleep(1)
detail1

100%|██████████| 700/700 [16:20<00:00,  1.40s/it]


[{'teamId': 200,
  'win': 'Win',
  'firstBlood': True,
  'firstTower': True,
  'firstInhibitor': True,
  'firstBaron': False,
  'firstDragon': True,
  'firstRiftHerald': True,
  'towerKills': 9,
  'inhibitorKills': 2,
  'baronKills': 0,
  'dragonKills': 4,
  'vilemawKills': 0,
  'riftHeraldKills': 1,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 111, 'pickTurn': 6},
   {'championId': 54, 'pickTurn': 7},
   {'championId': 360, 'pickTurn': 8},
   {'championId': 134, 'pickTurn': 9},
   {'championId': 64, 'pickTurn': 10}]},
 {'teamId': 100,
  'win': 'Fail',
  'firstBlood': True,
  'firstTower': False,
  'firstInhibitor': False,
  'firstBaron': False,
  'firstDragon': False,
  'firstRiftHerald': False,
  'towerKills': 0,
  'inhibitorKills': 0,
  'baronKills': 0,
  'dragonKills': 0,
  'vilemawKills': 0,
  'riftHeraldKills': 0,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 234, 'pickTurn': 1},
   {'championId': 350, 'pickTurn': 2},
   {'championId': 39, 'pickTurn': 3},
   {'

In [53]:
c = detail1

In [54]:
c

[{'teamId': 200,
  'win': 'Win',
  'firstBlood': True,
  'firstTower': True,
  'firstInhibitor': True,
  'firstBaron': False,
  'firstDragon': True,
  'firstRiftHerald': True,
  'towerKills': 9,
  'inhibitorKills': 2,
  'baronKills': 0,
  'dragonKills': 4,
  'vilemawKills': 0,
  'riftHeraldKills': 1,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 111, 'pickTurn': 6},
   {'championId': 54, 'pickTurn': 7},
   {'championId': 360, 'pickTurn': 8},
   {'championId': 134, 'pickTurn': 9},
   {'championId': 64, 'pickTurn': 10}]},
 {'teamId': 100,
  'win': 'Fail',
  'firstBlood': True,
  'firstTower': False,
  'firstInhibitor': False,
  'firstBaron': False,
  'firstDragon': False,
  'firstRiftHerald': False,
  'towerKills': 0,
  'inhibitorKills': 0,
  'baronKills': 0,
  'dragonKills': 0,
  'vilemawKills': 0,
  'riftHeraldKills': 0,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 234, 'pickTurn': 1},
   {'championId': 350, 'pickTurn': 2},
   {'championId': 39, 'pickTurn': 3},
   {'

In [57]:
from pymongo import MongoClient
conn = MongoClient('localhost', 27017)

# collection 삭제하고 다시 넣어야함 

In [62]:
# lol 데이터베스 생성
db = conn.lol

# score collection 생성
col_score = db.score

# team_score collection 생성 
col_team_score = db.team_score

# socre collection에 detail 추가 
col_score.insert_many(a)

# team_score collection에 detail 추가 
col_team_score.insert_many(c)

In [63]:
# DB에 잘 올라갔는지 확인 / 게임 승리시 챔피언 번호 가져오기

a = list(col_score.find({ 'stats.win': True }))
b = []
for i in a:
    b.append(i['championId'])

b.sort()
b
# a = list(col_score.find({ 'stats.win': True, 'championId': 35 })) #

[5,
 5,
 5,
 5,
 5,
 5,
 6,
 16,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 19,
 19,
 24,
 24,
 24,
 24,
 25,
 25,
 25,
 25,
 25,
 25,
 31,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 36,
 38,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 51,
 54,
 54,
 54,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 60,
 60,
 60,
 60,
 60,
 61,
 61,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 67,
 74,
 75,
 75,
 75,
 75,
 75,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 77,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 78,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 81,
 8

In [64]:
# ChampionId가 숫자로 되어 있어 이름으로 바꾸기 위해 api 활용
# apiDefault['key']

# 챔피언 이름 정보를 받아오기 위해 현재 사용 중인 버전 확인 필요
r = requests.get('https://ddragon.leagueoflegends.com/api/versions.json') # version data 확인
current_ver = r.json()[0] # 현재 최신 버전 확인
current_ver

'11.14.1'

In [65]:
r = requests.get('http://ddragon.leagueoflegends.com/cdn/{}/data/ko_KR/champion.json'.format(current_ver))
data = r.json()
cham_df = pd.DataFrame(data)
cham_df.head()


type             format  version  \
Aatrox   champion  standAloneComplex  11.14.1   
Ahri     champion  standAloneComplex  11.14.1   
Akali    champion  standAloneComplex  11.14.1   
Alistar  champion  standAloneComplex  11.14.1   
Amumu    champion  standAloneComplex  11.14.1   

                                                      data  
Aatrox   {'version': '11.14.1', 'id': 'Aatrox', 'key': ...  
Ahri     {'version': '11.14.1', 'id': 'Ahri', 'key': '1...  
Akali    {'version': '11.14.1', 'id': 'Akali', 'key': '...  
Alistar  {'version': '11.14.1', 'id': 'Alistar', 'key':...  
Amumu    {'version': '11.14.1', 'id': 'Amumu', 'key': '...

In [66]:
r = requests.get('http://ddragon.leagueoflegends.com/cdn/{}/data/ko_KR/summoner.json'.format(current_ver))
data = r.json()
spell_df = pd.DataFrame(data)
spell_df.head()

type  version  \
SummonerBarrier  summoner  11.14.1   
SummonerBoost    summoner  11.14.1   
SummonerDot      summoner  11.14.1   
SummonerExhaust  summoner  11.14.1   
SummonerFlash    summoner  11.14.1   

                                                              data  
SummonerBarrier  {'id': 'SummonerBarrier', 'name': '방어막', 'desc...  
SummonerBoost    {'id': 'SummonerBoost', 'name': '정화', 'descrip...  
SummonerDot      {'id': 'SummonerDot', 'name': '점화', 'descripti...  
SummonerExhaust  {'id': 'SummonerExhaust', 'name': '탈진', 'descr...  
SummonerFlash    {'id': 'SummonerFlash', 'name': '점멸', 'descrip...

In [67]:
# cham_df에 data 컬럼안에 필요한 정보가 있어 data 컬럼안에 내용을 꺼내서 기존의 cham_df와 대치 후 필요 없는 컬럼 삭제

# data 컬럼에 필요한 내용 추출
cham_list = {}
for i, cham in enumerate(cham_df.data):
    cham_list[i] = pd.Series(cham)
    
cham_df1 = pd.DataFrame(cham_list)

# cham_df와 같은 형태가 되도록 전치
cham_df = cham_df1.T

# 필요 없는 행 삭제(id, key, name 컬럼 제외 삭제)
cham_df.drop(['version', 'title', 'blurb', 'info', 'image', 'tags', 'partype', 'stats'], axis = 1, inplace = True)


In [68]:
cham_df.to_csv('/Users/bj/New Project/data/id.csv')

In [69]:
cham_df[cham_df['name'] == '뽀삐']

id key name
89  Poppy  78   뽀삐

In [70]:
r = requests.get('http://ddragon.leagueoflegends.com/cdn/{}/data/ko_KR/summoner.json'.format(current_ver))
data = r.json()
spell_df = pd.DataFrame(data)
spell_df.head()

type  version  \
SummonerBarrier  summoner  11.14.1   
SummonerBoost    summoner  11.14.1   
SummonerDot      summoner  11.14.1   
SummonerExhaust  summoner  11.14.1   
SummonerFlash    summoner  11.14.1   

                                                              data  
SummonerBarrier  {'id': 'SummonerBarrier', 'name': '방어막', 'desc...  
SummonerBoost    {'id': 'SummonerBoost', 'name': '정화', 'descrip...  
SummonerDot      {'id': 'SummonerDot', 'name': '점화', 'descripti...  
SummonerExhaust  {'id': 'SummonerExhaust', 'name': '탈진', 'descr...  
SummonerFlash    {'id': 'SummonerFlash', 'name': '점멸', 'descrip...

In [71]:
spell_list = {}
for i, spell in enumerate(spell_df.data):
    spell_list[i] = pd.Series(spell)
    
spell_df1 = pd.DataFrame(spell_list)

# # cham_df와 같은 형태가 되도록 전치
spell_df = spell_df1.T

# 필요 없는 행 삭제(id, key, name 컬럼 제외 삭제)
spell_df.drop(['description', 'tooltip', 'maxrank', 'cooldown', 'cooldownBurn','cost', 'costBurn',
               'datavalues', 'effect', 'effectBurn', 'vars', 'summonerLevel', 'modes', 'costType',
               'maxammo', 'range', 'rangeBurn', 'image', 'resource'], axis = 1, inplace = True)
spell_df = spell_df[['key', 'name', 'id']]

In [72]:
spell_df

key         name                            id
0   21          방어막               SummonerBarrier
1    1           정화                 SummonerBoost
2   14           점화                   SummonerDot
3    3           탈진               SummonerExhaust
4    4           점멸                 SummonerFlash
5    6          유체화                 SummonerHaste
6    7           회복                  SummonerHeal
7   13           총명                  SummonerMana
8   30       왕을 향해!            SummonerPoroRecall
9   31       포로 던지기             SummonerPoroThrow
10  11           강타                 SummonerSmite
11  39           표식  SummonerSnowURFSnowball_Mark
12  32           표식              SummonerSnowball
13  12         순간이동              SummonerTeleport
14  54  Placeholder  Summoner_UltBook_Placeholder

In [73]:
spell_df.to_csv('/Users/bj/New Project/data/spell.csv')

In [75]:
qq = []

qq.append(dict(cham_df.iloc[:,0]))
qq.append(dict(cham_df.iloc[:,1]))
qq.append(dict(cham_df.iloc[:,2]))

print(qq)

[{0: 'Aatrox', 1: 'Ahri', 2: 'Akali', 3: 'Alistar', 4: 'Amumu', 5: 'Anivia', 6: 'Annie', 7: 'Aphelios', 8: 'Ashe', 9: 'AurelionSol', 10: 'Azir', 11: 'Bard', 12: 'Blitzcrank', 13: 'Brand', 14: 'Braum', 15: 'Caitlyn', 16: 'Camille', 17: 'Cassiopeia', 18: 'Chogath', 19: 'Corki', 20: 'Darius', 21: 'Diana', 22: 'DrMundo', 23: 'Draven', 24: 'Ekko', 25: 'Elise', 26: 'Evelynn', 27: 'Ezreal', 28: 'Fiddlesticks', 29: 'Fiora', 30: 'Fizz', 31: 'Galio', 32: 'Gangplank', 33: 'Garen', 34: 'Gnar', 35: 'Gragas', 36: 'Graves', 37: 'Gwen', 38: 'Hecarim', 39: 'Heimerdinger', 40: 'Illaoi', 41: 'Irelia', 42: 'Ivern', 43: 'Janna', 44: 'JarvanIV', 45: 'Jax', 46: 'Jayce', 47: 'Jhin', 48: 'Jinx', 49: 'Kaisa', 50: 'Kalista', 51: 'Karma', 52: 'Karthus', 53: 'Kassadin', 54: 'Katarina', 55: 'Kayle', 56: 'Kayn', 57: 'Kennen', 58: 'Khazix', 59: 'Kindred', 60: 'Kled', 61: 'KogMaw', 62: 'Leblanc', 63: 'LeeSin', 64: 'Leona', 65: 'Lillia', 66: 'Lissandra', 67: 'Lucian', 68: 'Lulu', 69: 'Lux', 70: 'Malphite', 71: 'Malzaha

In [76]:
def IdToName(list):
    for i in range(len(list)):
        print(cham_df[cham_df['key'] == str(list[i])]['name'].values)

In [77]:
list(col_score.find({ 'stats.win': True, 'championId': 78 }))

[{'_id': ObjectId('60f661fab831580f0b9132d7'),
  'participantId': 3,
  'teamId': 100,
  'championId': 78,
  'spell1Id': 4,
  'spell2Id': 11,
  'stats': {'participantId': 3,
   'win': True,
   'item0': 6632,
   'item1': 2031,
   'item2': 3047,
   'item3': 3742,
   'item4': 1011,
   'item5': 3076,
   'item6': 3364,
   'kills': 4,
   'deaths': 5,
   'assists': 11,
   'largestKillingSpree': 3,
   'largestMultiKill': 1,
   'killingSprees': 1,
   'longestTimeSpentLiving': 684,
   'doubleKills': 0,
   'tripleKills': 0,
   'quadraKills': 0,
   'pentaKills': 0,
   'unrealKills': 0,
   'totalDamageDealt': 127974,
   'magicDamageDealt': 9341,
   'physicalDamageDealt': 98495,
   'trueDamageDealt': 20137,
   'largestCriticalStrike': 0,
   'totalDamageDealtToChampions': 8684,
   'magicDamageDealtToChampions': 614,
   'physicalDamageDealtToChampions': 7281,
   'trueDamageDealtToChampions': 788,
   'totalHeal': 7909,
   'totalUnitsHealed': 1,
   'damageSelfMitigated': 23281,
   'damageDealtToObjective

In [ ]:
# d점멸 f점멸 승률비교 

In [125]:
# 첫번째 용 먹었을 때 승률 비교

In [127]:
# 챔피언별 승률 비교

In [128]:
# 포지션별 승률비교 

In [129]:
# 게임 시간 별 승률 비교

In [130]:
# 상위 플레이어 플레이 통계와 비교 